In [1]:
!nvidia-smi    

Tue Jan 31 18:42:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip -q '/content/gdrive/ My Drive/cats_and_dogs_small-hw.zip' 


Import the necessary packages

In [4]:
import matplotlib
matplotlib.use("Agg")
import tensorflow as tf
from itertools import cycle
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import label_binarize
from sklearn.metrics import auc, roc_auc_score,classification_report,roc_curve
from tensorflow.keras import losses
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
import imutils
from google.colab.patches import cv2_imshow
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os



VGG16 Architecture

In [5]:
class VGG16:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
        # first set of CONV => RELU => POOL layers
        #block1
        model.add(Conv2D(64, (3, 3), padding="same", activation='relu', input_shape=inputShape))
        model.add(Conv2D(64, (3, 3), padding="same",activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        #block2
        model.add(Conv2D(128, (3, 3), padding="same", activation='relu', input_shape=inputShape))
        model.add(Conv2D(128, (3, 3), padding="same",activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        #block3
        model.add(Conv2D(256, (3, 3), padding="same", activation='relu', input_shape=inputShape))
        model.add(Conv2D(256, (3, 3), padding="same",activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        #block4
        model.add(Conv2D(512, (3, 3), padding="same", activation='relu', input_shape=inputShape))
        model.add(Conv2D(512, (3, 3), padding="same",activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        #block5
        model.add(Conv2D(512, (3, 3), padding="same", activation='relu', input_shape=inputShape))
        model.add(Conv2D(512, (3, 3), padding="same",activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(Dense(4096, activation='relu'))
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        # return the constructed network architecture
        return model

Mixup Augmentation

In [6]:
# For Beta distribution.
alpha = [0.2]
beta = [0.2]
def mixup(a, b):
  
  (image1, label1), (image2, label2) = a, b

  dist = tfd.Beta(alpha, beta)
  l = dist.sample(1)[0][0]
  
  img = l*image1+(1-l)*image2
  lab = l*label1+(1-l)*label2

  return img, lab


Cutmix Augmentation

In [7]:
import tensorflow_probability as tfp
tfd = tfp.distributions
import tensorflow as tf
IMG_SHAPE = 128
def get_bbox(l):
  cut_rat = tf.math.sqrt(1.-l)

  cut_w = IMG_SHAPE*cut_rat #rw
  cut_w = tf.cast(cut_w, tf.int32)
  
  cut_h = IMG_SHAPE*cut_rat #rh
  cut_h = tf.cast(cut_h, tf.int32)
  
  cx = tf.random.uniform((1,), minval=0, maxval=IMG_SHAPE, dtype=tf.int32) #rx
  cy = tf.random.uniform((1,), minval=0, maxval=IMG_SHAPE, dtype=tf.int32) #ry
  
  bbx1 = tf.clip_by_value(cx[0] - cut_w // 2, 0, IMG_SHAPE)
  bby1 = tf.clip_by_value(cy[0] - cut_h // 2, 0, IMG_SHAPE)
  bbx2 = tf.clip_by_value(cx[0] + cut_w // 2, 0, IMG_SHAPE)
  bby2 = tf.clip_by_value(cy[0] + cut_h // 2, 0, IMG_SHAPE)
  
  target_h = bby2-bby1
  if target_h ==0:
    target_h+=1

  target_w = bbx2-bbx1
  if target_w ==0:
    target_w+=1

  return bbx1, bby1, target_h, target_w

@tf.function
def cutmix(a, b):
  
  (image1, label1), (image2, label2) = a, b

  alpha = [1.]
  beta = [1.]
  
  ## Get sample from beta distribution
  dist = tfd.Beta(alpha, beta)
  ## Lambda
  l = dist.sample(1)[0][0]

  ## Get bbox ofsets and heights and widths
  bbx1, bby1, target_h, target_w = get_bbox(l)

  ## Get patch from image2
  crop2 = tf.image.crop_to_bounding_box(image2, bby1, bbx1, target_h, target_w)
  ## Pad the patch with same offset
  image2 = tf.image.pad_to_bounding_box(crop2, bby1, bbx1, IMG_SHAPE, IMG_SHAPE)
  ## Get patch from image1
  crop1 = tf.image.crop_to_bounding_box(image1, bby1, bbx1, target_h, target_w)
  ## Pad the patch with same offset
  img1 = tf.image.pad_to_bounding_box(crop1, bby1, bbx1, IMG_SHAPE, IMG_SHAPE)

  ## Subtract the patch from image1 so that patch from image2 can be put on instead
  image1 = image1-img1
  ## Add modified image1 and image2 to get cutmix image
  image = image1+image2

  ## Adjust lambda according to pixel ration
  l = 1 - (target_w * target_h) / (IMG_SHAPE * IMG_SHAPE)
  l = tf.cast(l, tf.float32)

  ## Combine labels
  label = l*label1+(1-l)*label2

  return image, label

Train, val, and test set
Preprocess image

In [8]:
plotpath= "/content/plot1.png"
datatrain= "/content/cats_and_dogs_small/train"
dataval="/content/cats_and_dogs_small/validation"
datatest="/content/cats_and_dogs_small/test"
# initialize the number of epochs to train for, initia learning rate,
# and batch size
EPOCHS = 300
INIT_LR = 1e-3
BS = 32
# initialize the data and labels
print("[INFO] loading images...")
traindata = []
trainlabels = []
valdata = []
vallabels = []
testdata = []
testlabels = []
# grab the image paths and randomly shuffle them
trainPaths = list(paths.list_images(datatrain))

for trainPath in trainPaths:
    # load the image, pre-process it, and store it in the data list
    trainimage = cv2.imread(trainPath)
    trainimage = cv2.resize(trainimage, (128, 128))
    #cv2_imshow(trainimage)
    #cv2.waitKey(0)

    trainimage = img_to_array(trainimage)
    traindata.append(trainimage)
    # extract the class label from the image path and update the
    # labels list
    trainlabel = trainPath.split(os.path.sep)[-2]
    trainlabel = 1 if trainlabel == "dogs" else 0
    trainlabels.append(trainlabel)
 # scale the raw pixel intensities to the range [0, 1]
trainX = np.array(traindata, dtype="float32") / 255.0
trainY = np.array(trainlabels)
#val
valPaths = list(paths.list_images(dataval))

testPaths = list(paths.list_images(datatest))

for testPath in testPaths:
    # load the image, pre-process it, and store it in the data list
    testimage = cv2.imread(testPath)
    testimage = cv2.resize(testimage, (128, 128))
    testimage = img_to_array(testimage)
    testdata.append(testimage)
    # extract the class label from the image path and update the
    # labels list
    testlabel = testPath.split(os.path.sep)[-2]
    testlabel = 1 if testlabel == "dogs" else 0
    testlabels.append(testlabel)
 # scale the raw pixel intensities to the range [0, 1]
testX = np.array(testdata, dtype="float32") / 255.0
testY = np.array(testlabels)
for valPath in valPaths:
    # load the image, pre-process it, and store it in the data list
    valimage = cv2.imread(valPath)
    valimage = cv2.resize(valimage, (128, 128))
    valimage = img_to_array(valimage)
    valdata.append(valimage)
    # extract the class label from the image path and update the
    # labels list
    vallabel = valPath.split(os.path.sep)[-2]
    vallabel = 1 if vallabel == "dogs" else 0
    vallabels.append(vallabel)
 # scale the raw pixel intensities to the range [0, 1]
valX = np.array(valdata, dtype="float32") / 255.0
valY = np.array(vallabels)
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
valY = to_categorical(valY, num_classes=2)

[INFO] loading images...


Train with Cutmix Augmentation

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
trainloader1 = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(4096)
trainloader2 = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(4096)

trainloader = tf.data.Dataset.zip((trainloader1, trainloader2))
trainloader = (trainloader.shuffle(4096).map(cutmix, num_parallel_calls=AUTO).batch(32).prefetch(AUTO))

valloader = tf.data.Dataset.from_tensor_slices((valX, valY))
valloader = (valloader.batch(32).prefetch(AUTO))
     
# initialize the model
print("[INFO] compiling model...")
model = VGG16.build(width=128, height=128, depth=3, classes=2)
opt=SGD(learning_rate=INIT_LR, momentum=0.9)
model.compile(loss=losses.BinaryCrossentropy(label_smoothing=0.1), optimizer=opt, metrics=["accuracy"])
print("[INFO] training network...")
H = model.fit(x=trainloader,validation_data=valloader, epochs=EPOCHS)
# prediction
predictions = model.predict(testX)
predictions=predictions.argmax(axis=-1)
class_labels = ['cats', 'dogs']
y_score = model.predict(testX, batch_size = 32) # one hot encoded softmax predictions
ytest_binary = label_binarize(testY, classes = [0,1,2]) # one hot encode the test data true labels
n_classes=2
fpr = dict()
tpr = dict()
roc_auc = dict() 
# compute fpr and tpr with roc_curve from the ytest true labels to the scores
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(ytest_binary[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# plot each class  curve on single graph for multi-class one vs all classification
colors = cycle(['blue', 'red', 'green'])
for i, color, lbl in zip(range(n_classes), colors, class_labels):
    plt.plot(fpr[i], tpr[i], color = color, lw = 1.5,
    label = 'ROC Curve of class {0} (area = {1:0.3f})'.format(lbl, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw = 1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for cats and dogs Data')
plt.legend(loc = 'lower right', prop = {'size': 6})
fullpath = '/content/roc_curve.png'
plt.savefig(fullpath)
plt.show()
# save the model to disk
print("[INFO] serializing network...")
model.save("/content/cats_and_dogs.h5", save_format="h5")
print(classification_report(testY, predictions))
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dogs/Cats")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plotpath)


Train with mixup augmentation

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
trainloader1 = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(4096)
trainloader2 = tf.data.Dataset.from_tensor_slices((trainX, trainY)).shuffle(4096)

trainloader = tf.data.Dataset.zip((trainloader1, trainloader2))
trainloader = (trainloader.shuffle(4096).map(mixup, num_parallel_calls=AUTO).batch(32).prefetch(AUTO))

valloader = tf.data.Dataset.from_tensor_slices((valX, valY))
valloader = (valloader.batch(32).prefetch(AUTO))
     
# initialize the model
print("[INFO] compiling model...")
model = VGG16.build(width=128, height=128, depth=3, classes=2)
opt=SGD(learning_rate=INIT_LR, momentum=0.9)
model.compile(loss=losses.BinaryCrossentropy(label_smoothing=0.1), optimizer=opt, metrics=["accuracy"])
# train the network
print("[INFO] training network...")
H = model.fit(x=trainloader,validation_data=valloader, epochs=EPOCHS)
# prediction
predictions = model.predict(testX)
predictions=predictions.argmax(axis=-1)
class_labels = ['cats', 'dogs']
y_score = model.predict(testX, batch_size = 32) # one hot encoded softmax predictions
ytest_binary = label_binarize(testY, classes = [0,1,2]) # one hot encode the test data true labels
n_classes=2
fpr = dict()
tpr = dict()
roc_auc = dict() 
# compute fpr and tpr with roc_curve from the ytest true labels to the scores
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(ytest_binary[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# plot each class  curve on single graph for multi-class one vs all classification
colors = cycle(['blue', 'red', 'green'])
for i, color, lbl in zip(range(n_classes), colors, class_labels):
    plt.plot(fpr[i], tpr[i], color = color, lw = 1.5,
    label = 'ROC Curve of class {0} (area = {1:0.3f})'.format(lbl, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw = 1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for cats and dogs Data')
plt.legend(loc = 'lower right', prop = {'size': 6})
fullpath = '/content/roc_curve.png'
plt.savefig(fullpath)
plt.show()
print("[INFO] serializing network...")
model.save("/content/cats_and_dogs.h5", save_format="h5")
print(classification_report(testY, predictions))
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dogs/Cats")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plotpath)

Train with not augmented

In [ ]:
# initialize the model
print("[INFO] compiling model...")
model = VGG16.build(width=128, height=128, depth=3, classes=2)
opt=SGD(learning_rate=INIT_LR, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
# train the network
print("[INFO] training network...")
H = model.fit(x=trainX, y=trainY,validation_data=(valX, valY), steps_per_epoch=len(trainX) // BS,epochs=EPOCHS, verbose=1)
# prediction
predictions = model.predict(testX)
predictions=predictions.argmax(axis=-1)
class_labels = ['cats', 'dogs']
y_score = model.predict(testX, batch_size = 32) # one hot encoded softmax predictions
ytest_binary = label_binarize(testY, classes = [0,1,2]) # one hot encode the test data true labels
n_classes=2
fpr = dict()
tpr = dict()
roc_auc = dict() 
# compute fpr and tpr with roc_curve from the ytest true labels to the scores
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(ytest_binary[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
# plot each class  curve on single graph for multi-class one vs all classification
colors = cycle(['blue', 'red', 'green'])
for i, color, lbl in zip(range(n_classes), colors, class_labels):
    plt.plot(fpr[i], tpr[i], color = color, lw = 1.5,
    label = 'ROC Curve of class {0} (area = {1:0.3f})'.format(lbl, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw = 1.5)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for cats and dogs Data')
plt.legend(loc = 'lower right', prop = {'size': 6})
fullpath = '/content/roc_curve.png'
plt.savefig(fullpath)
plt.show()
print("[INFO] serializing network...")
model.save("/content/cats_and_dogs.h5", save_format="h5")
print(classification_report(testY, predictions))
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dogs/Cats")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plotpath)

Evaluate and test model

In [ ]:
#Evaluating our Convolutional Neural Network image classifier
# grab the image paths and randomly shuffle them
#trainPaths = sorted(list(paths.list_images(datatest)))
#random.seed(42)
#random.shuffle(trainPaths)
#for trainPath in trainPaths:
# load the image, pre-process it, and store it in the data list
trainPath= '/content/111.jpg'
trainimage = cv2.imread(trainPath)
trainimage = cv2.imread(trainPath)
orig=trainimage.copy()

trainimage = cv2.resize(trainimage, (128, 128))
trainimage = img_to_array(trainimage)

image = np.expand_dims(trainimage, axis=0)
# load the trained convolutional neural network
print("[INFO] loading network...")
model = load_model("/content/cats_and_dogs.h5")
# classify the input image
(cats, dogs) = model.predict(image)[0]
# build the label
label = "dogs" if dogs > cats else "cats"
proba = dogs if dogs > cats else cats
label = "{}: {:.2f}%".format(label, proba * 100)
# draw the label on the image
output = imutils.resize(orig, width=400)
cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
    0.7, (0, 255, 0), 2)
# show the output image
cv2_imshow(output)
cv2.waitKey(0)



Grad-Cam Method

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Model
class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName
        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()
    def find_target_layer(self):
            # attempt to find the final convolutional layer in the network
            # by looping over the layers of the network in reverse order
            for layer in reversed(self.model.layers):
                # check to see if the layer has a 4D output
                if len(layer.output_shape) == 4:
                    return layer.name
            # otherwise, we could not find a 4D layer so the GradCAM
            # algorithm cannot be applied
            raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")
    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output,
                self.model.output])
    # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]
        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)
        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads
        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]
        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))
        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
        # return the resulting heatmap to the calling function
        return heatmap
    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap=cv2.COLORMAP_VIRIDIS):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)

Test images with Grad-Cam Method

In [ ]:
model= load_model("/content/cats_and_dogs.h5")
image="/content/111.jpg"
# load the original image from disk and then
# resize the image to its target dimensions
image = cv2.imread(image)
image = cv2.resize(image, (128, 128))
orig=image.copy()
# load the input image from disk (in Keras/TensorFlow format) and
# preprocess it
image = image.astype('float32')/255
image = np.expand_dims(image, axis=0)
# use the network to make predictions on the input image and find
# the class label index with the largest corresponding probability
preds = model.predict(image)
i = np.argmax(preds[0])
for idx in range(len(model.layers)):
  print(model.get_layer(index = idx).name)
# initialize our gradient class activation map and build the heatmap
cam = GradCAM(model, i, 'conv2d_9')
#print(cam)
heatmap = cam.compute_heatmap(image)
# resize the resulting heatmap to the original input image dimensions
# and then overlay heatmap on top of the image
heatmap = cv2.resize(heatmap, (128, 128))
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
fig, ax = plt.subplots(1, 3)
ax[0].imshow(heatmap)
ax[1].imshow(orig)
ax[2].imshow(output)



In [ ]:
plt.close('all')